Example of a wrapper for turning various user inputs into astropy SkyCoord objects following 

http://docs.astropy.org/en/stable/coordinates/



In [1]:
from navo_utils import utils

In [2]:
utils.parse_coords([10.625, 41.2])

[<SkyCoord (ICRS): (ra, dec) in deg
     (10.625, 41.2)>]

In [3]:
utils.parse_coords([[10.625, 41.2],[10.625, 41.2],[10.625, 41.2]])

[<SkyCoord (ICRS): (ra, dec) in deg
     (10.625, 41.2)>, <SkyCoord (ICRS): (ra, dec) in deg
     (10.625, 41.2)>, <SkyCoord (ICRS): (ra, dec) in deg
     (10.625, 41.2)>]

In [4]:
utils.parse_coords(['10.625', '41.2'])

[<SkyCoord (ICRS): (ra, dec) in deg
     (10.625, 41.2)>]

In [5]:
utils.parse_coords('10.625 41.2')

[<SkyCoord (ICRS): (ra, dec) in deg
     (10.625, 41.2)>]

In [6]:
utils.parse_coords(['00h42m30s', '+41d12m00s'])

[<SkyCoord (ICRS): (ra, dec) in deg
     (10.625, 41.2)>]

In [7]:
utils.parse_coords([['00h42m30s', '+41d12m00s'],['00h42m30s', '+41d12m00s']])

[<SkyCoord (ICRS): (ra, dec) in deg
     (10.625, 41.2)>, <SkyCoord (ICRS): (ra, dec) in deg
     (10.625, 41.2)>]

In [8]:
utils.parse_coords(['00h42.5m', '+41d12m'])

[<SkyCoord (ICRS): (ra, dec) in deg
     (10.625, 41.2)>]

In [9]:
utils.parse_coords([['00h42.5m', '+41d12m'],['00h42.5m', '+41d12m']])

[<SkyCoord (ICRS): (ra, dec) in deg
     (10.625, 41.2)>, <SkyCoord (ICRS): (ra, dec) in deg
     (10.625, 41.2)>]

In [10]:
utils.parse_coords('00 42 30 +41 12 00')

[<SkyCoord (ICRS): (ra, dec) in deg
     (10.625, 41.2)>]

In [11]:
utils.parse_coords(['00 42 30', '+41 12 00'])

[<SkyCoord (ICRS): (ra, dec) in deg
     (10.625, 41.2)>]

In [12]:
utils.parse_coords([['00 42 30', '+41 12 00'],['00 42 30', '+41 12 00']])

[<SkyCoord (ICRS): (ra, dec) in deg
     (10.625, 41.2)>, <SkyCoord (ICRS): (ra, dec) in deg
     (10.625, 41.2)>]

In [13]:
utils.parse_coords(['00:42.30','+41:12.00'])

[<SkyCoord (ICRS): (ra, dec) in deg
     (10.575, 41.2)>]

In [14]:
utils.parse_coords([['00:42.30','+41:12.00'],['00:42.30','+41:12.00']])

[<SkyCoord (ICRS): (ra, dec) in deg
     (10.575, 41.2)>, <SkyCoord (ICRS): (ra, dec) in deg
     (10.575, 41.2)>]